In [1]:
pip install selenium webdriver-manager pandas


Defaulting to user installation because normal site-packages is not writeable
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.6 MB 4.2 MB/s eta 0:00:03
   -------- ------------------------------- 2.1/9.6 MB 6.9 MB/s eta 0:00:02
   -------------- ------------------------- 3.4/9.6 MB 6.9 MB/s eta 0:00:01
   --------------------- ------------------ 5.2/9.6 MB 7.6 MB/s eta 0:00:01
   ----------------------------- ---------- 7.1/9.6 MB 7.6 MB/s eta 0:00:01
   ------------------------------------ --- 8.7/9.6 MB 7.9 MB/s eta 0:00:01
   ---------------------------------------- 9.6/9.6 MB 7.4 MB/s  0:00:01
Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)

   ------- --------------------------------  2/11 [wsproto]
   ------------------ ---------------------  5/11 [async-generator]
   --------------------- ------------------  6/11 [webdriver-manager]
   -------


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# Setup
options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    url = "https://cellphones.com.vn/mobile.html"
    driver.get(url)
    print("✓ Đã truy cập trang")
    time.sleep(5)
    
    # Đếm sản phẩm ban đầu
    products_initial = driver.find_elements(By.CSS_SELECTOR, '.product-info')
    print(f"✓ Số sản phẩm ban đầu: {len(products_initial)}")
    
    # Click button
    click_count = 0
    max_clicks = 100
    
    while click_count < max_clicks:
        try:
            # Scroll CHẬM xuống để button xuất hiện
            current_scroll = driver.execute_script("return window.pageYOffset;")
            driver.execute_script(f"window.scrollTo(0, {current_scroll + 800});")
            time.sleep(2)
            
            # Tìm TẤT CẢ button (bao gồm cả a tag, div...)
            all_buttons = driver.find_elements(By.XPATH, "//*[contains(text(), 'Xem thêm')]")
            
            print(f"\n--- Tìm thấy {len(all_buttons)} elements có 'Xem thêm' ---")
            
            # In ra TẤT CẢ
            product_button = None
            for idx, btn in enumerate(all_buttons):
                try:
                    btn_text = btn.text
                    btn_tag = btn.tag_name
                    btn_class = btn.get_attribute('class')
                    print(f"{idx+1}. Tag: {btn_tag}, Class: {btn_class}")
                    print(f"   Text: '{btn_text}'")
                    
                    # Tìm button có "sản phẩm" KHÔNG có "bình luận"
                    if 'sản phẩm' in btn_text.lower() and 'bình luận' not in btn_text.lower():
                        product_button = btn
                        print(f"   ✓✓✓ CHỌN BUTTON NÀY! ✓✓✓")
                except:
                    continue
            
            if not product_button:
                print("\n✗ Không tìm thấy button 'Xem thêm sản phẩm' lần này")
                print("  → Thử scroll thêm...")
                
                # Nếu đã scroll quá 3 lần mà không thấy thì dừng
                if click_count == 0:
                    # Chưa click lần nào, tiếp tục scroll tìm
                    time.sleep(2)
                    continue
                else:
                    # Đã click rồi mà không thấy nữa = hết
                    break
            
            # Click button
            print(f"\n✓ CLICK LẦN {click_count + 1}")
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", product_button)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", product_button)
            click_count += 1
            
            # Đợi load
            time.sleep(4)
            
            # Đếm sản phẩm
            products_current = driver.find_elements(By.CSS_SELECTOR, '.product-info')
            print(f"  → Tổng sản phẩm hiện tại: {len(products_current)}")
            
        except Exception as e:
            print(f"\n✓ Lỗi hoặc đã hết")
            print(f"  Chi tiết: {str(e)[:150]}")
            break
    
    # Kết quả cuối
    time.sleep(2)
    products_final = driver.find_elements(By.CSS_SELECTOR, '.product-info')
    print(f"\n{'='*50}")
    print(f"✓ TỔNG CLICK: {click_count}")
    print(f"✓ TỔNG SẢN PHẨM: {len(products_final)}")
    print(f"{'='*50}")
    
    driver.save_screenshot('step2_result.png')
    print("✓ Screenshot: step2_result.png")
    
    input("\nEnter để đóng...")
    
finally:
    driver.quit()


✓ Đã truy cập trang
✓ Số sản phẩm ban đầu: 35

--- Tìm thấy 4 elements có 'Xem thêm' ---
1. Tag: a, Class: button btn-show-more button__show-more-product
   Text: 'Xem thêm 910 sản phẩm'
   ✓✓✓ CHỌN BUTTON NÀY! ✓✓✓
2. Tag: a, Class: btn-show-more button__content-show-more
   Text: 'Xem thêm'
3. Tag: button, Class: btn-show-more button__cmt-showmore
   Text: 'Xem thêm 1555 bình luận'
4. Tag: script, Class: 
   Text: ''

✓ CLICK LẦN 1
  → Tổng sản phẩm hiện tại: 55

--- Tìm thấy 4 elements có 'Xem thêm' ---
1. Tag: a, Class: button btn-show-more button__show-more-product
   Text: 'Xem thêm 888 sản phẩm'
   ✓✓✓ CHỌN BUTTON NÀY! ✓✓✓
2. Tag: a, Class: btn-show-more button__content-show-more
   Text: 'Xem thêm'
3. Tag: button, Class: btn-show-more button__cmt-showmore
   Text: 'Xem thêm 1555 bình luận'
4. Tag: script, Class: 
   Text: ''

✓ CLICK LẦN 2
  → Tổng sản phẩm hiện tại: 75

--- Tìm thấy 4 elements có 'Xem thêm' ---
1. Tag: a, Class: button btn-show-more button__show-more-product
  

KeyboardInterrupt: 

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re

# Setup
options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

phones_data = []

try:
    url = "https://cellphones.com.vn/mobile.html"
    driver.get(url)
    print("✓ Đã truy cập trang")
    time.sleep(5)
    
    products = driver.find_elements(By.CSS_SELECTOR, '.product-info')
    print(f"✓ Số sản phẩm: {len(products)}")
    
    product_links = []
    for product in products:
        try:
            link_element = product.find_element(By.TAG_NAME, 'a')
            link = link_element.get_attribute('href')
            product_links.append(link)
        except:
            continue
    
    print(f"✓ Đã lấy {len(product_links)} links sản phẩm\n")
    
    for idx, link in enumerate(product_links[:5], 1):
        try:
            print(f"{'='*60}")
            print(f"[{idx}/{len(product_links[:5])}] Đang crawl: {link}")
            driver.get(link)
            time.sleep(3)
            
            phone_info = {
                'name': '',
                'storage': '',
                'chipset': '',
                'screen_size': '',
                'rating': '',
                'color': '',
                'price': '',
                'review_text': ''
            }
            
            # === BOX LEFT ===
            try:
                box_left = driver.find_element(By.CSS_SELECTOR, '.box-detail-product__box-left')
                
                # Name
                try:
                    name_element = box_left.find_element(By.CSS_SELECTOR, '.box-product-name h1')
                    phone_info['name'] = name_element.text.strip()
                    print(f"  ✓ Name: {phone_info['name']}")
                except:
                    print("  ✗ Name: Không lấy được")
                
                # Storage, Chipset, Screen Size
                try:
                    tech_items = box_left.find_elements(By.CSS_SELECTOR, '.technical-content-item')
                    for item in tech_items:
                        try:
                            tds = item.find_elements(By.TAG_NAME, 'td')
                            if len(tds) >= 2:
                                label = tds[0].text.lower().strip()
                                value = tds[1].text.strip()
                                
                                if 'bộ nhớ trong' in label:
                                    phone_info['storage'] = value
                                    print(f"  ✓ Storage: {value}")
                                elif 'chipset' in label or 'chip' in label:
                                    phone_info['chipset'] = value
                                    print(f"  ✓ Chipset: {value}")
                                elif 'kích thước màn hình' in label:
                                    phone_info['screen_size'] = value
                                    print(f"  ✓ Screen Size: {value}")
                        except:
                            continue
                except:
                    print("  ✗ Thông số kỹ thuật: Không lấy được")
                
                # Rating
                try:
                    rating_box = box_left.find_element(By.CSS_SELECTOR, '.box-rating')
                    rating_spans = rating_box.find_elements(By.TAG_NAME, 'span')
                    
                    rating_value = ''
                    total_rating = ''
                    
                    for span in rating_spans:
                        span_class = span.get_attribute('class') or ''
                        if 'total-rating' in span_class:
                            total_rating = span.text.strip()
                        elif span.text.strip() and 'total-rating' not in span_class:
                            rating_value = span.text.strip()
                    
                    phone_info['rating'] = f"{rating_value} sao/ {total_rating}"
                    print(f"  ✓ Rating: {phone_info['rating']}")
                except:
                    print("  ✗ Rating: Không lấy được")
                    
            except:
                print("  ✗ Box Left: Không tìm thấy")
            
            # === BOX CENTER ===
            try:
                box_center = driver.find_element(By.CSS_SELECTOR, '.box-detail-product__box-center')
                
                try:
                    active_variant = box_center.find_element(By.CSS_SELECTOR, '.item-variant.active')
                    
                    # Color
                    try:
                        color_element = active_variant.find_element(By.CSS_SELECTOR, '.item-variant-name')
                        phone_info['color'] = color_element.text.strip()
                        print(f"  ✓ Color: {phone_info['color']}")
                    except:
                        print("  ✗ Color: Không lấy được")
                    
                    # Price
                    try:
                        price_element = active_variant.find_element(By.CSS_SELECTOR, '.item-variant-price')
                        phone_info['price'] = price_element.text.strip()
                        print(f"  ✓ Price: {phone_info['price']}")
                    except:
                        print("  ✗ Price: Không lấy được")
                        
                except:
                    print("  ✗ Variant: Không tìm thấy")
                    
            except:
                print("  ✗ Box Center: Không tìm thấy")
            
            # === REVIEW TEXT (Sửa lại) ===
            try:
                print("  → Đang tìm reviews...")
                
                # Scroll xuống phần review để load
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
                
                # Thử nhiều selector khác nhau
                review_selectors = [
                    '#review .boxReview-comment-item-content',
                    '.boxReview-comment-item-content',
                    '#review .comment-content',
                    '.review-content'
                ]
                
                review_texts = []
                for selector in review_selectors:
                    try:
                        review_elements = driver.find_elements(By.CSS_SELECTOR, selector)
                        if review_elements:
                            print(f"  → Tìm thấy {len(review_elements)} reviews với selector: {selector}")
                            for elem in review_elements[:5]:
                                text = elem.text.strip()
                                # Loại bỏ các text không mong muốn
                                text = text.replace('Xem tất cả đánh giá', '').strip()
                                if text and len(text) > 10:  # Chỉ lấy review có nội dung
                                    review_texts.append(text)
                            break
                    except:
                        continue
                
                if review_texts:
                    phone_info['review_text'] = ' | '.join(review_texts)
                    print(f"  ✓ Reviews: Lấy được {len(review_texts)} reviews")
                else:
                    print("  ✗ Reviews: Không có nội dung")
                    
            except Exception as e:
                print(f"  ✗ Reviews: Lỗi - {str(e)[:80]}")
            
            phones_data.append(phone_info)
            print(f"  ✓ Hoàn thành sản phẩm {idx}\n")
            
        except Exception as e:
            print(f"  ✗ LỖI: {str(e)[:100]}\n")
            continue
    
    # Lưu CSV
    df = pd.DataFrame(phones_data)
    df.to_csv('cellphones_demo.csv', index=False, encoding='utf-8-sig')
    print(f"\n{'='*60}")
    print(f"✓ ĐÃ LƯU {len(phones_data)} SẢN PHẨM")
    print(f"{'='*60}")
    print("\n" + df.to_string())
    
    input("\nEnter để đóng...")
    
finally:
    driver.quit()


✓ Đã truy cập trang
✓ Số sản phẩm: 35
✓ Đã lấy 35 links sản phẩm

[1/5] Đang crawl: https://cellphones.com.vn/iphone-17-pro-max.html
  ✓ Name: iPhone 17 Pro Max 256GB | Chính hãng
  ✓ Screen Size: 6.9 inches
  ✓ Chipset: Chip A19 Pro
  ✓ Storage: 256 GB
  ✓ Rating: 5 sao/ (14 đánh giá)
  ✓ Color: Cam Vũ Trụ
  ✓ Price: 36.990.000₫
  → Đang tìm reviews...
  → Tìm thấy 5 reviews với selector: #review .comment-content
  ✓ Reviews: Lấy được 5 reviews
  ✓ Hoàn thành sản phẩm 1

[2/5] Đang crawl: https://cellphones.com.vn/iphone-17-256gb.html
  ✓ Name: iPhone 17 256GB | Chính hãng
  ✓ Screen Size: 6.3 inches
  ✓ Chipset: Apple A19
  ✓ Storage: 256 GB
  ✓ Rating: 5 sao/ (6 đánh giá)
  ✓ Color: Xanh Lam Khói
  ✓ Price: 24.190.000₫
  → Đang tìm reviews...
  → Tìm thấy 5 reviews với selector: #review .comment-content
  ✓ Reviews: Lấy được 5 reviews
  ✓ Hoàn thành sản phẩm 2

[3/5] Đang crawl: https://cellphones.com.vn/dien-thoai-samsung-galaxy-s25-ultra.html
  ✓ Name: Samsung Galaxy S25 Ultra 12G